## Lession 5: Dimensionality Reduction

**Prepared by John C.S. Lui** 

Date: March 2nd, 2021.

## Introduction

In this lesssion, we learn about techniques of reducing the number of features.  

Justifications for dimensionality reduction:
1. There may be *useless features* that have no use in our classification/regression.
2. Too many features may lead to a higher risk of overfitting.
3. Reduce computational cost in training.
4. Reduce the dimension to facility *visualization*

Two broad classes of dimensionality redution:
1. `Feature Selection`: Given $d$ features, want to reduce it to $k<d$ features
    and still have high accuracy in ML tasks.
2.  `Feature Projection`: Ususing statistical techniques to reduce number of features.
     Example techniques we plan to cover are **Principle Component Analysis (PCA)**, 
     **multidimensional scaling (MDS)**, and **Linear Discriminant Analysis (LDA)**.


In [ ]:
import sys
print(sys.version)

# We also need to define some functions which we will use

import os
CHART_DIR = "charts"  # define sub-directory to store charts (or figures)
if not os.path.exists(CHART_DIR):
    os.mkdir(CHART_DIR)  # create sub-directory
    
import matplotlib.pyplot as plt
plt.style.use('ggplot')  # use ggplot, a statistical visualization package similar to R

import numpy as np
import scipy

DPI = 300  # define the dot-per-inch for plotting

# define function to save file
def save_png(name):
    fn = 'Lesson_05_%s.png'%name 
    plt.savefig(os.path.join(CHART_DIR, fn), bbox_inches="tight")

## Feature Selection

If two features $X_i$ and $X_j$ are highly correlated, then having both of them in the ML
training may **NOT** help classification that much.   In statistics, we use **correlation** to
quantify the *LINEAR RELATIONSHIP* between two features.

One commone metric we often use is the **Pearson correlation coefficient** (also known
as PCC), which has a value between +1 and −1, where 1 is total 
*positive* linear correlation, 0 is no linear correlation, and −1 is 
total *negative* linear correlation. PCC between random variables $X$ and $Y$ is
$$
   \rho_{XY} = \frac{E[(X-\bar{X})(Y -\bar{Y}]}{\sigma_X \sigma_Y}
$$
where $\sigma_{Z}^{2} = E[(Z-\bar{Z})^2]$, and $\bar{Z}$ is the mean (or the first moment)
of the random variable $Z$.  Scipy has utility to help us to compute the PCC of two features.

In Scipy's pearsonr, for a given two equal-sized data series, it returns a **tuple** of 
two values, they are:
* the correlation coefficient value, and
* $p$-value:
The $p$-value describes how likely it is that the data series has been generated by an uncorrelated system. In other words, the **higher** the $p$-value, 
the **less we should trust** the correlation coefficient.   Let's demonstrate.

In [ ]:
from scipy.stats import pearsonr  # load pearsonr function

My_X = [1,2,3,4,5,6,7,8,9,10]
My_Y = [1,2,3,4,5,6,7,8,9,10.2]

print('My_X and My_Y are correlated because the PPC tuple is: \n', pearsonr(My_X, My_Y))

Your_X = [1,2,3,4,5,6,7,8,9,10]
Your_Y = [1, 20, 3,4, 5,6,7,8,9,10]
print('Your_X and Your_Y are not correlated because the PPC tuple is: \n', pearsonr(Your_X, Your_Y))



## Let's look at sample data

We generate $x$ from 0 to 10 with increment of 0.2. For y, we let it be x times 0.5 and 
then offset it by a normal distribution with $\mu=1$ and $\sigma = [0.01, 0.1, 1.0, 10.0]$.

In [ ]:
from scipy.stats import norm  # import normal distribution generator

def _plot_correlation_func(x, y):  # define function to generate plot for RVs x and your 
    r, p = pearsonr(x, y)
    plt.scatter(x, y, c='r', s=15)
    plt.title("Cor($X_1$, $X_2$) = %.3f, $r$= %.3f" %(r, p))
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")

    f1 = scipy.poly1d(np.polyfit(x, y, 1))  # do a degree-1 polynomial fit
    plt.plot(x, f1(x), "b--", linewidth=2);
    
np.random.seed(0)  # to reproduce the data later on
plt.clf()
plt.figure(num=None, figsize=(8, 8), dpi=DPI)

x = np.arange(0, 10, 0.2)  # generate X from 0.0 to 10.0 with increment 0.2

plt.subplot(221)
y = 0.5 * x + norm.rvs(loc=1, scale=.01, size=len(x)) #loc is the mean, scale is the SD
_plot_correlation_func(x, y)

plt.subplot(222)
y = 0.5 * x + norm.rvs(1, scale=.1, size=len(x))
_plot_correlation_func(x, y)

plt.subplot(223)
y = 0.5 * x + norm.rvs(1, scale=1, size=len(x))
_plot_correlation_func(x, y)

plt.subplot(224)
y = 0.5*x + norm.rvs(1, scale=10, size=len(x))
_plot_correlation_func(x, y)

plt.autoscale(tight=True)
plt.grid(True)
plt.tight_layout()

save_png("01_corr_demo")

## Observation

1.  For the first 3 plots, we can either drop $X_1$ or $X_2$
2.  For the last plot, we need to keep *BOTH FEATURES*.

**IMPORTANT NOTE**:  Notice that PPC only quanitify the **linear** correlation. When 
features are **non-linearly** correlated, we have a problem. Let's demonstrate.

In [ ]:
plt.figure(num=None, figsize=(8, 8), dpi=DPI)

x = np.arange(-5, 5, 0.2) # generate X from -5.0 to 5.0 with increment of 0.2

# Y = 0.5*X^2 plus some noise with is normally distributed
plt.subplot(221)
y = 0.5 * x ** 2 + norm.rvs(1, scale=.01, size=len(x))
_plot_correlation_func(x, y)

plt.subplot(222)
y = 0.5 * x ** 2 + norm.rvs(1, scale=.1, size=len(x))
_plot_correlation_func(x, y)

plt.subplot(223)
y = 0.5 * x ** 2 + norm.rvs(1, scale=1, size=len(x))
_plot_correlation_func(x, y)

plt.subplot(224)
y = 0.5 * x ** 2 + norm.rvs(1, scale=10, size=len(x))
_plot_correlation_func(x, y)

plt.autoscale(tight=True)
plt.grid(True)
plt.tight_layout()

save_png("02_corr_demo")

## Observation

1.  Features $X_1$ and $X_2$ are correlated, but the Pearson r value is low.
2.  If we do some *transformation*, say plot $X_1$ vs. $X_{1}^2$, then we can use PPC

Can we pick a better measure?

## Mutual Information

Let's consider **mutual Information** via **entropy**.

In simple terms, mutual information measures *how much information one feature ($X_2$)
provides, given that we already have another feature (say $X_1$)*.  

First, let us define the notion of *entropy* to 
measure the amount of information (or uncertainty) in a RV.  For a RV $X$, its entropy is:
$$   H(X) = - \sum_{\forall x} \mbox{Prob}\,[X=x] \log (\mbox{Prob}\,[X=x]) 
$$

**Example:** Let say $X$ is a r.v. denoting a Bernoulli random variable of $x \in \{0,1\}$
and $\mbox{Prob}[X=1]=p$ and $\mbox{Prob}[X=0]=1-p$.  Using the above formula for
entropy, when $p=0.5$, $H(X)=1.0$. In fact, The entropy has always between 0 to 1.
Let's illustrate $H(X)$ for different values of $p$.

In [ ]:
from scipy.stats import entropy

plt.clf()
plt.figure(num=None, figsize=(4, 4), dpi=DPI)

plt.title("Entropy $H(X)$")
plt.xlabel("$P(X=1)=p$")
plt.ylabel("$H(X)$")

plt.xlim(xmin=0, xmax=1.1)
x = np.arange(0.001, 1, 0.001)  # generate different value of $p$
y = -x * np.log2(x) - (1 - x) * np.log2(1 - x)
plt.plot(x, y, c='black')

plt.autoscale(tight=True)
plt.grid(True)
plt.ylim((0,1.05))
plt.xlim((-.01,1.01))

save_png('03_entropy_demo')

## Mutual Information

Mutual information between RVs $X$ and $Y$ is
$$
I(X;Y) = \sum_{\forall x} \sum_{\forall y} P(X=x, Y=y) \log \frac{P(X=x,Y=y)}{P(X=x)P(Y=y)}
$$

Often times, we want to work with the **normalized mutual information**, $N\!I(X;Y)$, 
which is
$$  N\!I(X;Y) = \frac{I(X;Y)}{H(X) + H(Y)}
$$

The implementation of the mutual information we can can be done by **binning** 
the feature values, and then calculating the fraction of values in each bin.   Let's illustrate.

In [ ]:
# define function to calculate mutual information
def normalized_mutual_info(x, y, bins=10):
    counts_xy, bins_x, bins_y = np.histogram2d(x, y, bins=(bins, bins))
    counts_x, bins = np.histogram(x, bins=bins)
    counts_y, bins = np.histogram(y, bins=bins)

    counts_xy += 1
    counts_x += 1
    counts_y += 1
    P_xy = counts_xy / np.sum(counts_xy)
    P_x = counts_x / np.sum(counts_x)
    P_y = counts_y / np.sum(counts_y)

    I_xy = np.sum(P_xy * np.log2(P_xy / (P_x.reshape(-1, 1) * P_y)))

    return I_xy / (entropy(counts_x) + entropy(counts_y))

def _plot_mi_func(x, y):
    mi = normalized_mutual_info(x, y)
    plt.scatter(x, y, s=15)
    plt.title("NI($X_1$, $X_2$) = %.3f" % mi)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    

np.random.seed(0)  # to reproduce the data later on
plt.clf()
plt.figure(num=None, figsize=(8, 8), dpi=DPI)

x = np.arange(0, 10, 0.2)

plt.subplot(221)
y = 0.5 * x + norm.rvs(1, scale=.01, size=len(x))
_plot_mi_func(x, y)

plt.subplot(222)
y = 0.5 * x + norm.rvs(1, scale=.1, size=len(x))
_plot_mi_func(x, y)

plt.subplot(223)
y = 0.5 * x + norm.rvs(1, scale=1, size=len(x))
_plot_mi_func(x, y)

plt.subplot(224)
y = 0.5*x + norm.rvs(1, scale=10, size=len(x))
_plot_mi_func(x, y)

plt.autoscale(tight=True)
plt.grid(True)
plt.tight_layout()

save_png('04_mi_demo_1')

So mutual information can help us to discover *linear relationship*.  How about *non-linear*?


In [ ]:
plt.clf()
plt.figure(num=None, figsize=(8, 8), dpi=DPI)

x = np.arange(-5, 5, 0.2) # generate x from -5 to 5 with increment of 0.2

# y = 0.5 * x^2 + noise, where noise is normally distributed
plt.subplot(221)
y = 0.5 * x ** 2 + norm.rvs(1, scale=.01, size=len(x))
_plot_mi_func(x, y)

plt.subplot(222)
y = 0.5 * x ** 2 + norm.rvs(1, scale=.1, size=len(x))
_plot_mi_func(x, y)

plt.subplot(223)
y = 0.5 * x ** 2 + norm.rvs(1, scale=1, size=len(x))
_plot_mi_func(x, y)

plt.subplot(224)
y = 0.5 * x ** 2 + norm.rvs(1, scale=10, size=len(x))
_plot_mi_func(x, y)

plt.autoscale(tight=True)
plt.grid(True)
plt.tight_layout()

save_png('05_mi_demo_2')

## Observation

So mutual information can be used to reveal **BOTH** *linear* and *non-linear* relationship between two features

So we can do the following:
1. Calculate the normalized mutual information for **all feature pairs**. For every
   pair with too high a value, which we would have to pre-determine this threshold, we 
   would then drop one of these two features.
2. For regression, we could drop the feature that has **too little mutual information** 
   with the targeted value.
   
This is ok for small number of features. If $d=1,000,000$, then we need to consider
$1000000 \times 999999$ pairs. Luckily, we can use various **feature projection methods**, like the **principle component analysis (PCA)**.

Another disadvantage is that we discard features that may not be that important in
isolation, but at times, we can find that some *combination of inputs* may determine 
the output. A good example is the XOR function wherein $X_1$ and $X_2$ are two input features.

## Using ML models to do feature selection (or filtering)

Another way people often use is to rely on the underlying ML model to do feature selection
via *filtering*.  This is known as **wrapper** and it can be achieved via the following:

1. Use features in X
2. Using model Y and input features in X to do the training
3. After step 2, examine the *importance* of individual feature in X
4. If the number of features is too big, then *filter* some least important ones from X, go to step 1.  Else, terminate.

In scikit-learn, there are various excellent wrapper classes in the `sklearn.feature_selection` package. A real workhorse in this field is
the *recursive feature elimination* (or `RFE`). It takes an estimator and the 
desired number of features to keep as parameters,  then trains the estimator with 
various feature sets. Let's illustrate.


In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import make_classification  # make_classification is a data generator

# we generate 100 samples, each with 10 features, informative feature is only 3
X, y = make_classification(n_samples=100, n_features=10, n_informative=3, random_state=0)

#print('X=', X)
#print('y=', y)

#print('Type of X=', type(X), '; dimension of X=', X.shape)
#print('Type of y=', type(y), '; dimension of y=', y.shape)

clf = LogisticRegression()    # create handle for LR
clf.fit(X, y)

selector = RFE(clf, n_features_to_select=3)  # call REF and state we need to select the top 3 features
selector = selector.fit(X, y)   # fit model to REF selector
print(selector.support_)    # print which feature to select
print(selector.ranking_)    # print ranking of each feature

In [ ]:
# Let's loop through to select 1,2,...,10 features to select, and see which features wil
# be selected

for i in range(1, 11):
    selector = RFE(clf, n_features_to_select=i)
    selector = selector.fit(X, y)
    print("i=%i\t support=%s\n\t ranking=%s" % (i, selector.support_, selector.ranking_))

## Observation

1. When we use RFE to select features, the results are *stable*: features which are kept
   when we set a lower threshold are also included when we increase the threshold
   
**Note**: During model training, we also use train/test set splitting to warn us 
when we are not doing correctly (e.g., low classification training/testing performance).
Also, many learners have the built-in feature selection (e.g., decision tree). Also,
learners can use regularization, say $L_1$ norm, to filter those features that are not
that important.


## Feature Projection Methods

At times, even when we apply feature selection, we still find the feature space is too
large, or when we need to reduce a huge feature space to 2 or 3 so to *visualize* it. We
often use the feature projection methods.  Let's go through some of them here:
1. Principal component analysis (**PCA**), which is an *unsupervised linear projection* method
2. Linear Discriminant Analysis (**LDA**), which is a *supervised linear projection* method
3. Multidimensional scaling (**MDS**), a *non-linear projection* method

# Principal Compnent Analysis (PCA)

Given the input $X$, PCA finds a linear projection of $X$ in a lower dimensional space 
such that it has the following properties:
1. The resulting variance is maximized
2. The final reconstruction error is minimized

PCA can be applied to classification or regressin problems, its basic algorithm is:
1. Given $X$, make sure all features are centered (by subtracting the feature's average)
2. Compute the covariance matrix
3. Compute the eigenvectors and eigenvalues of the covariance matrix

To demonstrate, assume that we start with $d = 1000$ features and our learner can
only handle no more than 20 features. So we pick the 20 eigenvectors with the **highest eigenvalues**.

In [ ]:
from sklearn import decomposition # get PCA from deomposition package

np.random.seed(3)  # pin down the random seed

#x1 = np.arange(0, 10, .2)  # generate x from 0 to 10 with increment of 0.2
#x2 = x1 + np.random.normal(scale=1, size=len(x1)) # x2 is like x1 with added noise

plt.clf()
fig = plt.figure(num=None, figsize=(10, 4), dpi=DPI)
plt.subplot(121)

plt.title("Original feature space")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")

x1 = np.arange(0, 10, .2)  # generate x from 0 to 10 with increment of 0.2
x2 = x1 + np.random.normal(scale=1, size=len(x1))  # x2 is like x1 with added noise

good = (x1 > 5) | (x2 > 5) # define array of good points (or class 1 points)
bad = ~good                # define array of bad points (or class 0 points)

plt.scatter(x1[good], x2[good], edgecolor="blue", facecolor="blue", s=15)
plt.scatter(x1[bad], x2[bad], edgecolor="red", facecolor="white", s=15)

plt.grid(True)

plt.subplot(122)

X = np.c_[(x1, x2)]

# setting parameters for PCA, where n_components is the # of features we want to reduce to
pca = decomposition.PCA(n_components=1)   # from d=2 to k=1
Xtrans = pca.fit_transform(X)

Xg = Xtrans[good]
Xb = Xtrans[bad]

plt.scatter(Xg[:, 0], np.zeros(len(Xg)), edgecolor="blue", facecolor="blue", s=15)
plt.scatter(Xb[:, 0], np.zeros(len(Xb)), edgecolor="red", facecolor="white", s=15)
plt.title("Transformed feature space")
plt.xlabel("$X'$")
fig.axes[1].get_yaxis().set_visible(False)

#print out the variance ratio of the first eigenvector
print('The variance ratio of the 1st eigenvector = ', pca.explained_variance_ratio_) 

plt.grid(True)

plt.autoscale(tight=True)
save_png("06_pca_demo")

## How to determine the number of dimension $k$ in scikit-learn?

Usually, we do not know what number of dimensions ($k$) is good. In that case, we leave the `n_components` parameter unspecified when initializing PCA, it will then calculate the 
**full transformation** (up to the original dimension $d$). After fitting the data,
`explained_variance_ratio_` contains an array of ratios in *decreasing order*: the 
first value is the ratio of the basis vector describing the direction of the *highest* variance, the second value is the ratio of the direction of the *second highest* variance,
and so on. 

Plots displaying the explained variance over the number of components are called 
**scree plots**. A nice example of combining a scree plot with a grid search to find the best setting for the classification problem can be found at http://scikit-learn.org/stable/auto_examples/plot_digits_pipe.html.

**Note**: In scikit-learn, there is a module `Kernel PCA`, which can do dimensionality
reduction for *non-linear project*.


## Limitation of PCA

Note that PCA does not use the label information to do dimensionality reduction.  So some times, PCA does the dimensionality reduction job, but it may make classification problem
a lot more difficult.  Let's illustrate this point.

In [ ]:
plt.clf()
fig = plt.figure(num=None, figsize=(10, 4), dpi=DPI)
plt.subplot(121)

plt.title("Original feature space\n (Class 1 when $X_1 > X_2$)")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")

x1 = np.arange(0, 10, .2)
x2 = x1 + np.random.normal(scale=1, size=len(x1))

good = x1 > x2  # define what are class 1 points
bad = ~good     # complement is class 2 points

plt.scatter(x1[good], x2[good], edgecolor="blue", facecolor="blue", s=15)
plt.scatter(x1[bad], x2[bad], edgecolor="red", facecolor="white", s=15)

plt.grid(True)

plt.subplot(122)  # prepare the 2nd plot

X = np.c_[(x1, x2)]

pca = decomposition.PCA(n_components=1)  # reduce it to 1 feature
Xtrans = pca.fit_transform(X)

Xg = Xtrans[good]
Xb = Xtrans[bad]

plt.scatter(Xg[:, 0], np.zeros(len(Xg)), edgecolor="blue", facecolor="blue", s=15)
plt.scatter(Xb[:, 0], np.zeros(len(Xb)), edgecolor="red", facecolor="white", s=15)
plt.title("Transformed feature space")
plt.xlabel("$X'$")
fig.axes[1].get_yaxis().set_visible(False) # turn off y-axis

print('The variance ratio of the 1st eigenvector = ', pca.explained_variance_ratio_) 


plt.grid(True)

plt.autoscale(tight=True)
save_png("07_pca_demo")

## Observation

Note that class 1 points are inter-mixed with class 0 points.  It becomes more difficult
to do classification !!!

## Linear Discriminant Analysis (LDA)

LDA is a linear supervised dimensionality reduction technique.  Unlike PCA, LDA uses
the label information.  Its objective is to do projection so that it can 
maximize the distance of points belonging to different classes, while minimizing the distances of points of the same class.  This way, after the projection, 
classification can be easily made.  Let's illustrate.

In [ ]:
# load the LDA module
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

plt.clf()
fig = plt.figure(num=None, figsize=(10, 4), dpi=DPI)
plt.subplot(121)

plt.title("Original feature space\n (Class 1 when $X_1 > X_2$)")
plt.xlabel("$X_1$")
plt.ylabel("$X_2$")

good = x1 > x2
bad = ~good

plt.scatter(x1[good], x2[good], edgecolor="blue", facecolor="blue", s=15)
plt.scatter(x1[bad], x2[bad], edgecolor="red", facecolor="white", s=15)

plt.grid(True)

plt.subplot(122)

X = np.c_[(x1, x2)]

lda_inst = LinearDiscriminantAnalysis(n_components=1) # instruct LDA to reduce to 1 dimension
Xtrans = lda_inst.fit_transform(X, good)  # we also provide labels to the fit_transform !!!

Xg = Xtrans[good]
Xb = Xtrans[bad]

plt.scatter(Xg[:, 0], np.zeros(len(Xg)), edgecolor="blue", facecolor="blue", s=15)
plt.scatter(Xb[:, 0], np.zeros(len(Xb)), edgecolor="red", facecolor="white", s=15)
plt.title("Transformed feature space")
plt.xlabel("$X'$")
fig.axes[1].get_yaxis().set_visible(False)

plt.grid(True)

plt.autoscale(tight=True)
save_png("08_lda_demo")

## Multidimensional Scaling (MDS)

MDS is a non-linear projection method, and its aim is to retain the 
*relative distances* among points as much as possible when reducing the dimensions. This 
is especially useful if we wan to *visualize* the data points at lower dimension. It applies
a distance function $f_d()$ (say Euclidean distance), and applies it as follows:

\begin{equation*}
\left( \begin{array}{ccc}
         x_{11} & \cdots & x_{1d} \\
         \vdots & \vdots & \vdots \\
         x_{N1} & \cdots & x_{Nd}
       \end{array}
\right)   \rightarrow 
\left( \begin{array}{ccc}
         f_d(X_1, X_1) & \cdots & f_d(X_1, X_N) \\
         \vdots & \vdots & \vdots \\
         f_d(X_N, X_1) & \cdots & f_d(X_N, X_N)
       \end{array}
\right)
\end{equation*}

Given this distance between any pair of points, MDS tries to position the individual 
data points in the lower dimensional space (usually two or three) such that the
new distance  resembles the distances in the original space as much as possible.
Let's illustrate.

In [ ]:
# Let's consider 3 points in 5-dimensional space
X = np.c_[np.ones(5), 2 * np.ones(5), 10 * np.ones(5)].T  # remember the transpose
print(X)

np.array([0,1,2,3,4])


In [ ]:
from sklearn import manifold, decomposition, datasets
from mpl_toolkits.mplot3d import Axes3D

np.random.seed(3) # for renegeration

# all examples will have three classes in this file
colors = ['g', 'r', 'b', 'k','c']
markers = ['o', 'P', 'X', 'D','s']

# create five points in which each point is 5-dimensional
X = np.c_[np.ones(5), 2 * np.ones(5), np.array([0,3,2,2,4]), 
          np.array([7,4,5,6,4]), 10 * np.ones(5)].T
y = np.array([0, 1, 2, 3, 4])

fig = plt.figure(figsize=(10, 4), dpi=DPI)

ax = fig.add_subplot(121, projection='3d')
ax.set_facecolor('lightgrey')

mds = manifold.MDS(n_components=3)  # MDS and ask to reduce to 3 features
Xtrans = mds.fit_transform(X)

for cl, color, marker in zip(np.unique(y), colors, markers):
    ax.scatter(
        Xtrans[y == cl][:, 0], Xtrans[y == cl][:, 1], Xtrans[y == cl][:, 2], c=color, marker=marker, edgecolor='black')
plt.title("MDS on example data set\n in 3 dimensions\n")
ax.view_init(10, -15)

mds = manifold.MDS(n_components=2)
Xtrans = mds.fit_transform(X)

ax = fig.add_subplot(122)
for cl, color, marker in zip(np.unique(y), colors, markers):
    ax.scatter(
        Xtrans[y == cl][:, 0], Xtrans[y == cl][:, 1], c=color, marker=marker, edgecolor='lightgrey')
plt.title("MDS on example data set\n in 2 dimensions")

save_png("09_mds_demo_1")

## MDS Visualization of the Iris dataset

Let's consider the Iris dataset which contains four attributes per data point. We will
project it into three-dimensional space while keeping the relative distances between 
the individual flowers as much as possible. If we do not specify any metric, so MDS will default to Euclidean.  Let's proceed.

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

# MDS

fig = plt.figure(figsize=(10, 4), dpi=DPI)

ax = fig.add_subplot(121, projection='3d')
ax.set_facecolor('lightgrey')

mds = manifold.MDS(n_components=3)
Xtrans = mds.fit_transform(X)

for cl, color, marker in zip(np.unique(y), colors, markers):
    ax.scatter(
        Xtrans[y == cl][:, 0], Xtrans[y == cl][:, 1], Xtrans[y == cl][:, 2], c=color, marker=marker, edgecolor='black')
plt.title("MDS on Iris data set\n in 3 dimensions\n")
ax.view_init(10, -15)

mds = manifold.MDS(n_components=2)
Xtrans = mds.fit_transform(X)

ax = fig.add_subplot(122)
for cl, color, marker in zip(np.unique(y), colors, markers):
    ax.scatter(
        Xtrans[y == cl][:, 0], Xtrans[y == cl][:, 1], c=color, marker=marker, edgecolor='black')
plt.title("MDS on Iris data set in 2 dimensions")

save_png("10_mds_demo_iris.png")

# PCA

fig = plt.figure(figsize=(10, 4), dpi=DPI)

ax = fig.add_subplot(121, projection='3d')
ax.set_facecolor('lightgrey')

pca = decomposition.PCA(n_components=3)
Xtrans = pca.fit(X).transform(X)

for cl, color, marker in zip(np.unique(y), colors, markers):
    ax.scatter(
        Xtrans[y == cl][:, 0], Xtrans[y == cl][:, 1], Xtrans[y == cl][:, 2], c=color, marker=marker, edgecolor='black')
plt.title("PCA on Iris data set \nin 3 dimensions\n")
ax.view_init(50, -35)

pca = decomposition.PCA(n_components=2)
Xtrans = pca.fit_transform(X)

ax = fig.add_subplot(122)
for cl, color, marker in zip(np.unique(y), colors, markers):
    ax.scatter(Xtrans[y == cl][:, 0], Xtrans[y == cl][:, 1], c=color, marker=marker, edgecolor='black')
plt.title("PCA on Iris data set in 2 dimensions")
plt.tight_layout()

save_png("11_pca_demo_iris")

## Observation and remark
1. For PCA, it has a larger expected  spread of flowers belonging to the same class.
2. MDS, by deault, use Euclidean distance function, but you can define your own.
3. For categorical data, one *cannot* use Euclidean distance, so you have to devicee
   your own distance function.
4. Both MDS and PCA are not a single algorithm, but rather a **family of different algorithms**. Please refer to the documentation

## Conclusion

We have discussed:
1. Feature selection vs. feature projection methods
2. How to use correlation, in particular, Pearson Coefficient, to find out linear 
   relationship among two features
3. Discuss how to use mutual information to discover linear and non-linear relations between two features.
4. Discuss how to use *recursive wrapper* to select features.
5. Discuss PCA, LDA and Multidimensioal Scaling (MDS).
6. There are other dimensionality reduction techniques, for example,
   **Isomap**, a variant of MDS, which tries to match distances between data but instead
   of Euclidean distance, it uses geodesic distances.  Another one is 
   **Laplacian eigenmaps**, which tries to match similarities between data,


<br>
<span style="font-family:times; font-size:0.9em;">
p.s.: The above codes are the "modified" and "enhanced" version of the codes from the book, "Building Machine Learning Systems with Python".</span>